In [ ]:
import geemap
import ee

ee.Initialize()

In [1]:
Map = geemap.Map()
Map.add_basemap('HYBRID')

    # Define widgets
    style = {'description_width': 'initial'}
    
    speed = widgets.IntSlider(description='  Frames per second:', tooltip='Frames per second:', value=1, min=1, max=5, style=style)
    start_year = widgets.IntSlider(description='Start Year:', value=1995, min=1984, max=2023, style=style)
    end_year = widgets.IntSlider(description='End Year:', value=2023, min=1984, max=2023, style=style)
    start_month = widgets.IntSlider(description='Start Month:', value=1, min=1, max=12, style=style)
    end_month = widgets.IntSlider(description='End Month:', value=12, min=1, max=12, style=style)
    nd_options = [
        'Vegetation Index (NDVI)',
        'Water Index (NDWI)',
        'Modified Water Index (MNDWI)',
        'Snow Index (NDSI)',
        'Soil Index (NDSI)',
        'Burn Ratio (NBR)',
    ]
    nd_indices = widgets.Dropdown(options=nd_options, value=None, description='NDI:', style=style)
    create_gif = widgets.Button(description='Submit', button_style='primary', tooltip='Click to create timelapse', style=style)
    output = widgets.Output()

    # Define widget layouts
    sidebar_box = widgets.VBox([button_box, widgets.Label("Page 1")])
    inputs_box = widgets.VBox([speed, start_year, end_year, start_month, end_month, nd_indices, create_gif, output])
    hbox = widgets.HBox([sidebar_box, inputs_box])
    
    # Button click handlers and map functionality
    first_band = None
    second_band = None

    def nd_index_change(change):
        nonlocal first_band
        nonlocal second_band
        if nd_indices.value == 'Burn Ratio (NBR)':
            first_band = 'NIR'
            second_band = 'SWIR2'
        elif nd_indices.value in [
            'Modified Water Index (MNDWI)',
            'Snow Index (NDSI)',
        ]:
            first_band = 'Green'
            second_band = 'SWIR1'
        elif nd_indices.value == 'Soil Index (NDSI)':
            first_band = 'SWIR1'
            second_band = 'NIR'
        elif nd_indices.value == 'Vegetation Index (NDVI)':
            first_band = 'NIR'
            second_band = 'Red'
        elif nd_indices.value == 'Water Index (NDWI)':
            first_band = 'NIR'
            second_band = 'SWIR1'

    nd_indices.observe(nd_index_change, names='value')

    def submit_clicked(b):
        with output:
            output.clear_output()
            if start_year.value > end_year.value:
                print('The end year must be greater than the start year.')
                return
            if start_month.value > end_month.value:
                print('The end month must be greater than the start month.')
                return
            add_progress_bar = start_year.value != end_year.value
            start_date = f'{str(start_month.value).zfill(2)}-01'
            end_date = f'{str(end_month.value).zfill(2)}-30'

            print('Computing...')

            nd_bands = None
            if (first_band is not None) and (second_band is not None):
                nd_bands = [first_band, second_band]

            Map.add_landsat_ts_gif(
                roi=Map.user_roi,
                label="GreenSat",
                start_year=start_year.value,
                end_year=end_year.value,
                start_date=start_date,
                end_date=end_date,
                bands="SWIR1/NIR/Red".split('/'),
                font_color="white",
                frames_per_second=speed.value,
                font_size=30,
                add_progress_bar=add_progress_bar,
                progress_bar_color="blue",
                download=True,
                apply_fmask=True,
                nd_bands=nd_bands,
                nd_threshold=0,
                nd_palette=["black", "blue"],
            )

    create_gif.on_click(submit_clicked)

IndentationError: unexpected indent (2667191643.py, line 2)